In [1]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
from dataclasses import dataclass

C:\Users\roshi\AppData\Local\Temp\ipykernel_312\1998135547.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
os.chdir("../")
%pwd

'c:\\Users\\roshi\\OneDrive\\Desktop\\Git-1\\Text_Classification_ML_NLP'

In [3]:
@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_tfidf: Path
    test_y: Path
    model_path: Path
    metrics_file_path: Path

In [4]:
from src.text_Classification_ML.constants import *
from src.text_Classification_ML.utils.common import *

In [5]:
class ConfigurationManager:
    def __init__(
            self,
            config_file_path = Config_File_Path
            ):
        self.config = read_yaml(config_file_path)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir= config.root_dir,
            test_tfidf= config.test_tfidf,
            test_y= config.test_y,
            model_path= config.model_path,
            metrics_file_path= config.metrics_file_path
        )

        return model_evaluation_config

In [6]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import joblib
import scipy.sparse
from src.text_Classification_ML.utils.common import *

In [7]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def metrics_eval(self, actual, pred):
        accuracy = accuracy_score(actual, pred)
        precision = precision_score(actual, pred, average= "micro")
        recall = recall_score(actual, pred, average= "micro")
        f1 = f1_score(actual, pred, average= "micro")

        return accuracy, precision, recall, f1
    
    def save_results(self):
        test_tfidf = scipy.sparse.load_npz(self.config.test_tfidf)
        test_y = pd.read_csv(self.config.test_y)

        test_label = test_y['Rating']

        model = joblib.load(self.config.model_path)

        y_pred = model.predict(test_tfidf)

        (accuracy, precision,recall,f1) = self.metrics_eval(test_label, y_pred)

        scores = {
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1_score": f1
        }

        save_json(path= Path(self.config.metrics_file_path), data= scores)

In [8]:
try:
    config= ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config= model_evaluation_config)
    model_evaluation.save_results()
except Exception as e:
    raise e